In [1]:
import warnings
warnings.filterwarnings('ignore')
from numpy import *
import numpy as np
import pandas as pd
import sys, os
import torch
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from GCFNN import GCFNN
import torch.nn.functional as F
from numpy.random import randint

In [2]:
def concatenated_tensor(X_set, M):
    concatenated_tensor_list = []
    # 遍历索引范围为 0 到 M-1 的值
    for i in range(M.shape[1]):
        # 拼接 X_set[i]
        concatenated_tensor_list.append(X_set[i])
        ## 将 adj_list[i] 转换为密集张量并拼接
        #adj_dense_tensor = adj_list[i].to_dense()
        #concatenated_tensor_list.append(adj_dense_tensor)
        
    # 将 tr_M 转换为张量并拼接
    M_tensor = torch.from_numpy(M)
    concatenated_tensor_list.append(M_tensor)
    # 沿着 dim=1 拼接所有张量
    concatenated_tensor = torch.cat(concatenated_tensor_list, dim=1)
    return concatenated_tensor

In [3]:
# read data
meta_data=pd.read_csv('meta.csv',index_col=0)
metagenomics=pd.read_csv('metagenomics.csv',index_col=0)
data_class={'nonIBD':0,'CD':1,'UC':1}
meta_data['diagnosis']=meta_data['diagnosis'].map(data_class)

x1=metagenomics
print('输入各组学数据表：',x1.shape,meta_data.shape)

#重构数据表和标签表
X1 = x1
index_new=X1.index
print(index_new)
                                        
meta_data=meta_data.reindex(index_new)
X1=array(X1.reindex(index_new))
print('集合所有样本重构数据表：',X1.shape,meta_data.shape)
X_set=[X1]
M        = len(X_set)
Mask     = np.ones([np.shape(X_set[0])[0], M])
#生成表示组学缺失情况的掩码矩阵，将缺失组学部分填补为0
for m_idx in range(M):
    Mask[np.isnan(X_set[m_idx]).all(axis=1), m_idx] = 0
    #X_set[m_idx][Mask[:, m_idx] == 0] = np.mean(X_set[m_idx][Mask[:, m_idx] == 1], axis=0)
    X_set[m_idx][Mask[:, m_idx] == 0] = 0
    
#对宏基因组数据预处理
def log(base,x):
    return np.log(x)/np.log(base)
X_set[0]=log(2,2*X_set[0]+0.00001)

#print(Mask)
#print(X_set)
Y = meta_data['diagnosis'].tolist()
Y = np.array(Y)
Y_onehot = np.zeros((Y.shape[0], 2))
Y_onehot[np.arange(Y.shape[0]), Y] = 1
print('标签转换为one-hot编码：', Y_onehot.shape, np.sum(Y_onehot[:, 0]))

输入各组学数据表： (1638, 578) (2891, 1)
Index(['PSM7J1BF', 'CSM7KORK', 'HSM7J4HS', 'MSM9VZNL', 'MSMB4LYB', 'MSM6J2PM',
       'MSMA26ET', 'CSM79HMN', 'CSM67UB1', 'HSM7J4IS',
       ...
       'HSM5FZBQ_P', 'MSM6J2IE', 'HSM7CYZ9', 'CSM7KOPG', 'ESM5MEAB_P',
       'ESM5MEB9_P', 'PSMA266M', 'CSM9X22K', 'CSM7KOP2', 'HSM7CZ36'],
      dtype='object', name='External ID', length=1638)
集合所有样本重构数据表： (1638, 578) (1638, 1)
标签转换为one-hot编码： (1638, 2) 429.0


In [4]:
# 指定保存模型的文件夹路径
folder_path = 'model_shap_mg'
# 创建文件夹
os.makedirs(folder_path, exist_ok=True)

cv = StratifiedKFold(n_splits=5,shuffle = True,random_state = 50)
auc_cv = []
acc_cv = []
f1_cv = []
for i,(train_index, test_index) in enumerate(cv.split(X_set[0],np.argmax(Y_onehot,axis=1))):
    tr_X_set, te_X_set, va_X_set = {}, {}, {}
    for m in range(len(X_set)):
        tr_X_set[m],tr_Y_onehot,tr_M = X_set[m][train_index],Y_onehot[train_index],Mask[train_index]
        te_X_set[m],te_Y_onehot,te_M = X_set[m][test_index],Y_onehot[test_index],Mask[test_index]
        #归一化
        #print(te_Y_onehot)
    def Normalize(data):
        """
        :param data:Input data
        :return:normalized data
        """
        mean = np.mean(data)
        mx = np.max(data)
        mn = np.min(data)
        return mean, mx, mn
    
    for m in range(M):
        mean, mx, mn = Normalize(tr_X_set[m])
        tr_X_set[m] = (tr_X_set[m] - mean) / (mx - mn)
        te_X_set[m] = (te_X_set[m] - mean) / (mx - mn)
    
    x_dim_set    = [tr_X_set[m].shape[1] for m in range(len(tr_X_set))]
    y_dim        = np.shape(tr_Y_onehot)[1]
    
    input_dims = {
        'x_dim_set': x_dim_set,
        'y_dim': y_dim,
        'z_dim': 50,
        #'steps_per_batch': steps_per_batch
    }
    network_settings = {
        'dim_enc': 100,
        'num_layers_enc': 3,   #view-specific
        'dim_specificpre': 100,
        'num_layers_specificpre': 2,  #multi-view
        'dim_joint_pre': 100,
        'num_layers_jointpre': 2,
        'dropout': 0.5,
        'edge_per_node': 8, 
        'ITERATION': 2000
    }
    
    for m in range(M):
        tr_X_set[m] = torch.from_numpy(tr_X_set[m])
        te_X_set[m] = torch.from_numpy(te_X_set[m])
    tr_Y_onehot = torch.from_numpy(tr_Y_onehot)
    te_Y_onehot = torch.from_numpy(te_Y_onehot)
    #tansform
    tr_input = concatenated_tensor(tr_X_set, tr_M)
    te_input = concatenated_tensor(te_X_set, te_M)
    GPU_use = True
    model = GCFNN(input_dims, network_settings, cuda = GPU_use)
    if GPU_use: 
        model = model.cuda()
        tr_input = tr_input.cuda()
        te_input = te_input.cuda()
        tr_Y_onehot = tr_Y_onehot.cuda()
        te_Y_onehot = te_Y_onehot.cuda()
    model_save, maxf1, maxacc, maxauc = model.train_model(train_input = tr_input, test_input = te_input, alpha = 1.0, beta = 0.01, l_rate = 0.0005, tr_Y_onehot = tr_Y_onehot, te_Y_onehot = te_Y_onehot)
    model_path = os.path.join(folder_path, 'model_'+str(i)+'.pth')
    torch.save(model_save, model_path)
    f1_cv.append(maxf1)
    acc_cv.append(maxacc)
    auc_cv.append(maxauc)
print('f1', f1_cv)
print('average f1: {}±{}'.format(np.mean(f1_cv), np.std(f1_cv)))
print('acc', acc_cv)
print('average acc: {}±{}'.format(np.mean(acc_cv), np.std(acc_cv)))
print('auc', auc_cv)
print('average auc: {}±{}'.format(np.mean(auc_cv), np.std(auc_cv)))

Train F1: 0.8897
00050: TRAIN| LT=0.818 LP=0.366 LKL=7.126 LPS=0.302 LKLS=7.979 | 
Test F1: 0.9535
Test ACC: 0.9299
Test AUC: 0.9562
Train F1: 0.9845
00100: TRAIN| LT=0.351 LP=0.100 LKL=8.247 LPS=0.068 LKLS=10.009 | 
Test F1: 0.9897
Test ACC: 0.9848
Test AUC: 0.9885
Train F1: 0.9856
00150: TRAIN| LT=0.259 LP=0.060 LKL=7.521 LPS=0.046 LKLS=7.700 | 
Test F1: 0.9834
Test ACC: 0.9756
Test AUC: 0.9909
Train F1: 0.9886
00200: TRAIN| LT=0.236 LP=0.055 LKL=7.186 LPS=0.044 LKLS=6.512 | 
Test F1: 0.9876
Test ACC: 0.9817
Test AUC: 0.9929
Train F1: 0.9912
00250: TRAIN| LT=0.235 LP=0.058 LKL=7.029 LPS=0.046 LKLS=6.176 | 
Test F1: 0.9876
Test ACC: 0.9817
Test AUC: 0.9930
Train F1: 0.9902
00300: TRAIN| LT=0.218 LP=0.047 LKL=7.068 LPS=0.038 LKLS=6.256 | 
Test F1: 0.9876
Test ACC: 0.9817
Test AUC: 0.9938
Train F1: 0.9897
00350: TRAIN| LT=0.225 LP=0.053 LKL=6.824 LPS=0.047 LKLS=5.710 | 
Test F1: 0.9855
Test ACC: 0.9787
Test AUC: 0.9937
Train F1: 0.9886
00400: TRAIN| LT=0.201 LP=0.043 LKL=6.761 LPS=0.035